In [1]:
!pip install datasets transformers accelerate evaluate torchtext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Importing Dependencies

In [2]:
#!pip install --force-reinstall numpy==1.23.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 92.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.1
    Uninstalling numpy-2.2.1:
      Successfully uninstalled numpy-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.23.4 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.23.4 which is incompatible.
bigframes 1.31.0 requires numpy>=1.24.0, but you have numpy 1.23.4 which is incompatible.
chex 0.1.88 requires numpy>=1.24.1, but you have numpy 1.23.4 which is incompatible.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.12.0 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.4 which is incompatible.
jax 0.4.33 requires numpy>=

In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

## Loading Dataset

In [6]:
imdb = load_dataset("imdb")

## Load Tokenizer & Model

In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Tokenize Data

In [9]:
def tokenize_function(example):
  return tokenizer(example["text"], truncation=True)

tokenize_imdb = imdb.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

## Data Collator

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Metric

In [17]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)

  return metric.compute(predictions=predictions, references=labels)


## Training Arguments

In [18]:
training_args = TrainingArguments(
    output_dir = "./results",
    evaluation_strategy = "epoch",
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs=3,
    fp16 = True,
    push_to_hub = False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Trainer

In [19]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenize_imdb["train"],
    eval_dataset = tokenize_imdb["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

<ipython-input-19-30779e2168bb>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Train The Model

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.101500,0.314264,0.917520
2,0.087000,0.340085,0.927320
3,0.028900,0.400158,0.930480


TrainOutput(global_step=4689, training_loss=0.08152724369982205, metrics={'train_runtime': 1356.2146, 'train_samples_per_second': 55.301, 'train_steps_per_second': 3.457, 'total_flos': 9834539051060448.0, 'train_loss': 0.08152724369982205, 'epoch': 3.0})

## Save Model

In [21]:
trainer.save_model("./my_sentiment_model")

In [24]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="./my_sentiment_model", tokenizer=tokenizer)
result = classifier("This movie was absolutely amazing!")
print(result)

Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.999607503414154}]


In [22]:
import shutil
import os
from google.colab import files

def zip_and_download(folder_path):
  """Zips a folder and downloads it using google.colab.files.download."""
  zip_file_path = f"{os.path.basename(folder_path)}.zip"
  shutil.make_archive(os.path.splitext(zip_file_path)[0], 'zip', folder_path)
  files.download(zip_file_path)

folder_to_download = "/content/my_sentiment_model"
zip_and_download(folder_to_download)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
import shutil
import os
from google.colab import files

def zip_and_download(folder_path):
  """Zips a folder and downloads it using google.colab.files.download."""
  zip_file_path = f"{os.path.basename(folder_path)}.zip"
  shutil.make_archive(os.path.splitext(zip_file_path)[0], 'zip', folder_path)
  files.download(zip_file_path)

folder_to_download = "/content/results"
zip_and_download(folder_to_download)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>